*INE*   
*Antonio Coín Castro*


In [1]:
import numpy as np
import pandas as pd

# Práctica 5 - PageRank y sistemas de recomendación

## PageRank

Calculamos PageRank en el subgrafo web formado por las páginas accesibles desde (e incluyendo) [esta dirección](http://arantxa.ii.uam.es/~abellogin/ir/C.html).

### Ejercicio 1.
*Tomar r = 0.1, N = nº de páginas en el subgrafo, y 0.3 como valor inicial de PageRank para empezar a iterar.*

Consideramos la matriz Google vista en clase:

$$
G = r\left(M + \frac{1}{n}ae^T \right) + (1-r)\left( \frac{1}{n}ee^T\right),
$$

donde $M$ es la matriz de hiperenlaces (normalizada por filas), $a$ un vector que contiene un $1$ en la posición $i$-ésima si la fila $i$-ésima de $M$ es nula, y un $0$ si no lo es; y $e$ es un vector de 1s.

Después, calculamos el vector de PageRank $\pi^*$ mediante la fórmula iterativa

$$
\pi^{(k+1)T} = \pi^{(k)T}G.
$$

In [2]:
def iterate_pagerank(M, r, v_init, v_pref=None, normalize=False,
                     max_iter=100, eps=1e-4, verbose=False):
    n = M.shape[0]
    ones = np.ones(n)

    if normalize:
        v0 = v_init/v_init.sum()
    else:
        v0 = v_init

    # Convert M to G
    S = M.copy()
    S[S.sum(axis=1) == 0] = n*[1./n]

    if v_pref is None:
        E = ones.reshape(-1, 1)@ones.reshape(1, -1)/n
    else:
        E = ones.reshape(-1, 1)@v_pref.reshape(1, -1)

    G = r*S + (1 - r)*E

    # Iterate PageRank
    v_old = v0
    if verbose:
        print(f"[Inicial]     v = {v0}")
    for it in range(max_iter):
        v_pagerank = v_old.T@G

        if verbose:
            print(f"[Iteración {it + 1}] v = {v_pagerank}")

        if np.linalg.norm(v_old - v_pagerank, ord=1) < eps:
            break

        v_old = v_pagerank

    return v_pagerank, it + 1

In [3]:
M = np.array([  # hyperlinks matrix
    [0, 1/2., 0, 0, 1/2.],      # A
    [0, 0, 0, 0, 0],            # B
    [1/3., 0, 0, 1/3., 1/3.],   # C
    [0, 0, 1., 0, 0],           # D
    [0, 1., 0, 0, 0]            # E
])
r = 0.1
n = M.shape[0]
v_init = np.full(n, 0.3)

pagerank, it = iterate_pagerank(M, r, v_init, verbose=True)
print(f"Converge en {it} iteraciones")

[Inicial]     v = [0.3 0.3 0.3 0.3 0.3]
[Iteración 1] v = [0.286 0.321 0.306 0.286 0.301]
[Iteración 2] v = [0.28662 0.32082 0.30502 0.28662 0.30092]
[Iteración 3] v = [0.28658373 0.3208394  0.3050784  0.28658373 0.30091473]
[Iteración 4] v = [0.28658607 0.32083745 0.30507516 0.28658607 0.30091525]
Converge en 4 iteraciones


In [4]:
webs = ['A', 'B', 'C', 'D', 'E']
print("PageRank")
for i, web in enumerate(webs):
    print(f"PR({web}): {pagerank[i]:.4f}")

PageRank
PR(A): 0.2866
PR(B): 0.3208
PR(C): 0.3051
PR(D): 0.2866
PR(E): 0.3009


Un detalle a tener en cuenta es que lo normal es interpretar el vector inicial como una *distribución de probabilidad*, de forma que sus elementos sean no negativos y sumen $1$. Dada la naturaleza del algoritmo, se mantienen estas propiedades en el vector final de PageRank. De esta forma, podemos interpretar este último como el porcentaje de tiempo que un *surfista aleatorio* pasaría en cada una de las páginas consideradas.

Damos la opción de normalizar el vector de entrada en el código (vemos que simplemente cambia la escala del resultado final).

In [5]:
pagerank_norm, it = iterate_pagerank(M, r, v_init, normalize=True)
print("PageRank normalizado")
for i, web in enumerate(webs):
    print(f"PR({web}): {pagerank_norm[i]:.4f}")

PageRank normalizado
PR(A): 0.1911
PR(B): 0.2139
PR(C): 0.2034
PR(D): 0.1911
PR(E): 0.2006


### Ejercicio 2.
*Con los mismos parámetros del apartado anterior, personalizar PageRank para un usuario que tiene sólo las páginas 'A.html' y 'D.html' entre sus preferencias.*

En este caso usamos la propuesta original de Brin & Page, que consiste en modificar la matriz de teleportación $E$ para dejarla como $E=ev^T$, donde $v$ es el *vector de personalización*. Este vector es una distribución de probabilidad que indica las preferencias del usuario de moverse a una u otra página. En el caso de que esta sea la distribución uniforme, recuperamos la matriz $E$ original.

En nuestro caso, como el usuario solo tiene preferencia por las páginas A y D (podemos asumir que tiene la misma preferencia por ambas), el vector de personalización es:
$$
v^T = (1/2, 0, 0, 1/2, 0).
$$

In [6]:
v_pref = np.array([1/2., 0, 0, 1/2., 0])
pagerank_pers, it = iterate_pagerank(M, r, v_init, v_pref, verbose=True)
print(f"Converge en {it} iteraciones")

[Inicial]     v = [0.3 0.3 0.3 0.3 0.3]
[Iteración 1] v = [0.691 0.051 0.036 0.691 0.031]
[Iteración 2] v = [0.67722 0.03867 0.07012 0.67722 0.03677]
[Iteración 3] v = [0.67811073 0.0383114  0.0684954  0.67811073 0.03697173]
[Iteración 4] v = [0.67804941 0.03836894 0.0685773  0.67804941 0.03695494]
[Iteración 5] v = [0.67805329 0.03836534 0.06857232 0.67805329 0.03695576]
Converge en 5 iteraciones


In [7]:
webs = ['A', 'B', 'C', 'D', 'E']
print("PageRank personalizado")
for i, web in enumerate(webs):
    print(f"PR({web}): {pagerank_pers[i]:.4f}")

PageRank personalizado
PR(A): 0.6781
PR(B): 0.0384
PR(C): 0.0686
PR(D): 0.6781
PR(E): 0.0370


### Ejercicio opcional.
*Probar valores distintos de r para calcular PageRank.*

In [8]:
webs = ['A', 'B', 'C', 'D', 'E']
rs = [0.0, 0.25, 0.5, 0.75, 1.0]

for r in rs:
    pagerank, it = iterate_pagerank(M, r, v_init)
    print(f"PageRank r={r} (converge en {it} iteraciones)")
    for i, web in enumerate(webs):
        print(f"PR({web}): {pagerank[i]:.4f}")

PageRank r=0.0 (converge en 1 iteraciones)
PR(A): 0.3000
PR(B): 0.3000
PR(C): 0.3000
PR(D): 0.3000
PR(E): 0.3000
PageRank r=0.25 (converge en 5 iteraciones)
PR(A): 0.2684
PR(B): 0.3516
PR(C): 0.3097
PR(D): 0.2684
PR(E): 0.3019
PageRank r=0.5 (converge en 8 iteraciones)
PR(A): 0.2421
PR(B): 0.4020
PR(C): 0.3112
PR(D): 0.2421
PR(E): 0.3026
PageRank r=0.75 (converge en 12 iteraciones)
PR(A): 0.2196
PR(B): 0.4515
PR(C): 0.3074
PR(D): 0.2196
PR(E): 0.3019
PageRank r=1.0 (converge en 19 iteraciones)
PR(A): 0.2000
PR(B): 0.5000
PR(C): 0.3000
PR(D): 0.2000
PR(E): 0.3000


Vemos que conforme aumenta el valor de $r$ aumenta el número de iteraciones necesarias para alcanzar la convergencia. Además, con $r=0$ observamos por ejemplo que el PageRank es el mismo para todas las páginas, y conforme aumenta $r$ va aumentando $PR(B)$. Esto no es de extrañar, ya que conforme $r$ aumenta más relevante es la matriz $S$, y en este matriz la columna correspondiente a $B$ es la que tiene la suma más alta.

In [9]:
webs = ['A', 'B', 'C', 'D', 'E']
rs = [0.0, 0.25, 0.5, 0.75, 1.0]

for r in rs:
    pagerank_pers, it = iterate_pagerank(M, r, v_init, v_pref)
    print(f"PageRank personalizado r={r} (converge en {it} iteraciones)")
    for i, web in enumerate(webs):
        print(f"PR({web}): {pagerank_pers[i]:.4f}")

PageRank personalizado r=0.0 (converge en 2 iteraciones)
PR(A): 0.7500
PR(B): 0.0000
PR(C): 0.0000
PR(D): 0.7500
PR(E): 0.0000
PageRank personalizado r=0.25 (converge en 6 iteraciones)
PR(A): 0.5800
PR(B): 0.1000
PR(C): 0.1500
PR(D): 0.5800
PR(E): 0.0900
PageRank personalizado r=0.5 (converge en 9 iteraciones)
PR(A): 0.4366
PR(B): 0.2161
PR(C): 0.2399
PR(D): 0.4366
PR(E): 0.1707
PageRank personalizado r=0.75 (converge en 6 iteraciones)
PR(A): 0.3115
PR(B): 0.3500
PR(C): 0.2861
PR(D): 0.3115
PR(E): 0.2409
PageRank personalizado r=1.0 (converge en 19 iteraciones)
PR(A): 0.2000
PR(B): 0.5000
PR(C): 0.3000
PR(D): 0.2000
PR(E): 0.3000


En el caso de personalización también aumenta el número de iteraciones conforme aumenta $r$. Sin embargo, en este caso con $r=0$ el PageRank se reparte únicamente entre las páginas favoritas, mientras que conforme va aumentando $r$ esta preferencia se va disipando poco a poco, llegando con $r=1$ al mismo vector que en el caso uniforme.

## Sistemas de recomendación

Usaremos [este dataset](https://docs.google.com/spreadsheets/d/1YmCY-T0ByZRXz8DpaFyieGfsS2jvxsprZr5dsSh8mpE) que se ha obtenido a partir de las puntuaciones que cada estudiante ha otorgado a un total de 150 películas y series. Las puntuaciones se miden en una escala de 1 a 5 con saltos de 0.5 puntos, mientras que 0 significa que la película no se ha visto.

### Ejercicio 1.

*Aplicar un algoritmo de filtrado colaborativo basado en memoria para generar recomendaciones de películas para cada estudiante. La estrategia a usar será a elección de cada estudiante, teniendo en cuenta las consideraciones vistas en clase. Aplicarlo utilizando las puntuaciones de entrenamiento como base para recomendar películas del conjunto de test, y observar el nivel de acierto (visualmente, comparando puntuación real de test y predicción del recomendador).*

In [10]:
data_train = pd.read_csv(
    "data/training-matrix.dat",
    header=0,
    sep='\t'
)
data_train.fillna(0, inplace=True)
X_train = data_train.to_numpy()[:, 1:]

In [11]:
data_test = pd.read_csv(
    "data/test-matrix.dat",
    header=0,
    sep='\t'
)
data_test.fillna(0, inplace=True)
y_test = data_test.to_numpy()[:, 1:]
X_test = y_test.copy()
X_test[X_test > 0] = 1

Elegimos hacer k-NN basado en items, es decir, calculamos las predicciones de cada usuario para cada ítem como:

$$
\hat r(u, x)= \sum_{y \in I(u)\cap V_k(x)} \operatorname{sim}(x, y)r(u, y),
$$

donde $\operatorname{sim}$ es la similitud entre items, $r$ es la puntuación de un ítem por parte de un usuario (conocida), $I(u)$ son los ítems que ha puntuado el usuario $u$, $V_k(x)$ es el $k$-vecindario (definido por similitud) del ítem $x$, y finalmente

$$
c=\frac{1}{\displaystyle \sum_{y \in I(u)\cap B(x, k)} \left|\operatorname{sim}(x, y)\right|}.
$$

Como función de similaridad consideramos dos variantes:

- *Coseno*:
$$
\operatorname{sim}(x, y) = \cos(r(x), r(y))=\frac{r(x)r(y)^T}{\|r(x)\|r(y)|\|},
$$
donde $r(\cdot) = (r(u_1, \cdot), \dots, r(u_N, \cdot))$ es el vector de recomendaciones de un ítem por parte de todos los usuarios.

- *Correlación de Pearson*:
$$
\operatorname{sim}(x, y) = \cos(r(x) - \bar r(x)e, r(y) - \bar r(y)e),
$$
donde $\bar r(\cdot)$ es la media de recomendaciones de un ítem, y $e$ es un vector de 1s. Notamos que para un $x$ e $y$ concretos, solo se calcula esta métrica considerando **los usuarios que hayan puntuado ambos ítems**.

Desarrollamos una clase que encapsule todo el procesamiento.

In [12]:
class ItemKNN():
    def __init__(self, k, similarity='cosine'):
        self.k = k
        self.similarity = similarity
        self.similarity_matrix = None
        self.ratings = None
        self.neighbours = {}

        if self.similarity == 'cosine':
            self.sim = self._cosine
        else:
            self.sim = self._pearson

    def _cosine(self, x, y):
        rx = self.ratings[x, :]
        ry = self.ratings[y, :]
        sim = rx@ry/(np.linalg.norm(rx)*np.linalg.norm(ry))
        u_xy = (rx*ry > 0).sum()  # number of users that rated x and y
        c = u_xy/50
        return c*sim

    def _pearson(self, x, y):
        users_rated_x = self.ratings[x, :] > 0
        users_rated_y = self.ratings[y, :] > 0
        users_rated_xy = users_rated_x & users_rated_y
        u_xy = users_rated_xy.sum()  # number of users that rated x and y

        if u_xy > 0:
            rx = self.ratings[x, users_rated_xy]
            ry = self.ratings[y, users_rated_xy]
            rx_centered = rx - np.mean(rx)
            ry_centered = ry - np.mean(ry)
            if np.linalg.norm(rx_centered)*np.linalg.norm(ry_centered) == 0:
                sim = 0  # one of the centered vectors is zero
            else:
                sim = (rx_centered@ry_centered
                       / (np.linalg.norm(rx_centered)*np.linalg.norm(ry_centered)))
        else:
            sim = 0

        c = u_xy/50
        return c*sim

    def fit(self, X):
        n_items = X.shape[0]
        self.ratings = X.copy()

        # Compute similarity matrix
        self.similarity_matrix = np.array(
            [[self.sim(x, y)
              for x in range(n_items)] for y in range(n_items)])

        # Compute all neighbours ordered by similarity
        for x in range(X.shape[0]):  # for each item
            all_neighbours = self.similarity_matrix[x].argsort()[::-1]
            for u in range(X.shape[1]):  # for each user
                rated_neighbours = all_neighbours[X[all_neighbours, u] > 0]
                self.neighbours[(x, u)] = rated_neighbours[1:self.k + 1]

        return self

    def predict(self, X):
        """X is > 0 in the positions to predict and 0 elsewhere"""
        if self.ratings is None:
            raise Exception('Model was not fitted.')

        preds = X.T.copy()  # n_users x n_items_rated
        for u in range(X.shape[1]):  # for each user
            scores_user = X[:, u]
            for x in range(X.shape[0]):
                item = scores_user[x]
                if item == 0:  # only predict non-zero items
                    continue

                neighbours = self.neighbours[(x, u)]
                sim_neighbours = self.similarity_matrix[x, neighbours]
                c = np.linalg.norm(sim_neighbours, 1)
                pred = (sim_neighbours@self.ratings[neighbours, u]/c
                        if c > 0 else np.mean(self.ratings))
                preds[u, x] = np.clip(pred, 1, 5)

        return preds.T  # n_items_rated x n_users

Hacemos una serie de consideraciones sobre el algoritmo implementado:

- Por defecto se utiliza la similaridad coseno. En el caso de elegir similaridad Pearson, los valores de $\bar r(\cdot)$ se calculan solo sobre los usuarios que han puntuado los ítems involucrados, en lugar de sobre todo el perfil.
- Multiplicamos las similitudes $\operatorname{sim}(x, y)$ por $|U(x)\cap U(y)|/50$, para penalizar aquellas con poca base de comparación. Recordamos que $U(\cdot)$ representa el conjunto de usuarios que han puntuado un ítem en concreto.
- Si $U(x)\cap U(y)=\emptyset$, se establece la similitud correspondiente en $0$.
- Para cada ítem y usuario tomamos como conjunto de vecinos los $k$ más similares que además **tengan rating del usuario en cuestión**. Así nos aseguramos que siempre existen vecinos (ya que hay una buena cantidad de ratings en el conjunto de datos).
- Si las predicciones se salen de rango, las truncamos a los extremos del intervalo $[1, 5]$. Además, para ítems que tengan similaridad 0 con todos sus vecinos, elegimos establecer la recomendación a la media global de todos los ratings.

Pasamos a entrenar nuestro modelo con la similaridad por defecto. Como tenemos 150 ítems, elegimos como un valor razonable de $k=20$ como tamaño del vecindario.

In [13]:
model = ItemKNN(k=20)
model.fit(X_train)
y_hat = model.predict(X_test)

Podemos observar empíricamente la calidad de las predicciones para un par de usuarios al azar.

In [14]:
us = [5, 10]
for u in us:
    gt = y_test[:, u]
    preds = y_hat[:, u]
    print(f"Ground truth usuario {u}:\n", gt[gt > 0])
    print(f"Predicciones usuario {u}:\n", preds[preds > 0])
    print()

Ground truth usuario 5:
 [2.  3.  5.  3.  4.  4.  1.  1.  2.  3.5 1.  3.  4.5 2.  3.5 3.  4.5 3.
 2. ]
Predicciones usuario 5:
 [3.13207784 3.14668932 3.35850722 3.83260254 3.68483143 3.37117935
 3.45395448 3.37220592 3.13635081 3.21074827 2.96162836 3.38481287
 3.05848508 3.40221463 2.88398343 3.3457418  3.35797811 3.36776469
 3.33648827]

Ground truth usuario 10:
 [3.5 5.  5.  4.5 2.  4.5 5.  4.5 1.  2.  3.5 2.  2.5 1.5 4.5 3. ]
Predicciones usuario 10:
 [3.7659531  3.6225076  3.56259288 3.73992553 3.70631536 3.95134126
 4.00707473 3.83532472 3.82513862 3.65803378 3.80952993 3.90194366
 3.86902892 3.76169233 3.724872   3.50069905]



Vemos que las predicciones son más o menos acertadas (no difieren en general demasiado de los valores reales), si bien son bastante mejorables.

### Ejercicio 2

*Calcular el MAE y RMSE de las recomendaciones producidas. Contrastar los resultados (MAE y RMSE) de dos estrategias diferentes de recomendación (o de la misma estrategia con diferentes parámetros).*

Las métricas de error MAE (Mean Absolute Error) y RMSE (Root Mean Squared Error) se definen como:

$$
MAE = \frac{1}{|R_{test}|}\sum_{(u, x)\in R_{test}} |\hat r(u, x) - r_{test}(u, x)|.
$$

$$
RMSE = \sqrt{\frac{1}{|R_{test}|}\sum_{(u, x)\in R_{test}} \left(\hat r(u, x) - r_{test}(u, x)\right)^2}.
$$

In [15]:
def compute_error_metrics(y_hat, y_test):
    gt = y_test[y_test > 0].flatten()
    preds = y_hat[y_hat > 0].flatten()
    R_test = len(gt)

    mae = np.linalg.norm(gt - preds, 1)/R_test
    mse = np.sqrt(np.linalg.norm(gt - preds, 2)**2/R_test)

    return mae, mse

Estudiamos el error con con varios modelos, aumentando progresivamente el valor de $k$.

In [16]:
ks = [1, 5, 10, 20, 50, 75, 100, 150]
for k in ks:
    model = ItemKNN(k=k)
    model.fit(X_train)
    y_hat = model.predict(X_test)
    mae, rmse = compute_error_metrics(y_hat, y_test)
    print(f"{k}-NN: MAE = {mae:.4f}, RMSE = {rmse:.4f}")

1-NN: MAE = 1.1488, RMSE = 1.5484
5-NN: MAE = 0.9671, RMSE = 1.2256
10-NN: MAE = 0.9250, RMSE = 1.1644
20-NN: MAE = 0.9031, RMSE = 1.1379
50-NN: MAE = 0.9063, RMSE = 1.1337
75-NN: MAE = 0.9080, RMSE = 1.1291
100-NN: MAE = 0.9094, RMSE = 1.1287
150-NN: MAE = 0.9099, RMSE = 1.1286


Como vemos, un valor de $k=1$ proporciona el mayor error en ambos casos, y el error con $k=5$ es el segundo peor también en ambos casos. Vemos que tomar un vecindario muy grande (100 ó 150 vecinos) hace que disminuya el RMSE, pero sin embargo el MAE desciende más si tomamos un tamaño intermedio (20 ó 50 vecinos).

En cuanto al análisis en términos absolutos, como ambos errores se miden en las mismas unidades que los ratings, podemos decir que en media estamos cometiendo un error en torno a $1$ en las predicciones, lo cual dependiendo de la aplicación concreta de nuestro sistema podría o no ser admisible.

Finalmente, podemos tomar un par de valores de $k$ y estudiar cómo se comporta el error con la métrica de similaridad basada en el coeficiente de Pearson.

In [17]:
similarity = ['cosine', 'pearson']
ks = [10, 20]
for k in ks:
    for sim in similarity:
        model = ItemKNN(k=k, similarity=sim)
        model.fit(X_train)
        y_hat = model.predict(X_test)
        mae, rmse = compute_error_metrics(y_hat, y_test)
        print(f"{k}-NN + {sim}: MAE = {mae:.4f}, RMSE = {rmse:.4f}")

10-NN + cosine: MAE = 0.9250, RMSE = 1.1644
10-NN + pearson: MAE = 0.9557, RMSE = 1.2291
20-NN + cosine: MAE = 0.9031, RMSE = 1.1379
20-NN + pearson: MAE = 0.9533, RMSE = 1.2121


Como vemos, el error es en general más alto con la similaridad Pearson.

### Ejercicio 3

*Calcular P@N (precisión) y R@N (recall) de las recomendaciones producidas a distintos N, considerando como relevantes aquellos ítems con un rating mayor que 3 en test. Para ello, tened en cuenta que habrá que predecir una puntuación (para cada usuario) usando todos los ítems del sistema excepto aquellos que ya ha visto en entrenamiento, ordenando dichas puntuaciones para generar el ranking.*

Si $\mathcal U$ es el conjunto de usuarios, las fórmulas para precisión y recall en este contexto son:

$$
P@N = \frac{1}{|\mathcal U|}\sum_{u\in\mathcal U} \frac{\left|Rel_u@N\right|}{|Rec_u@N|}.
$$

$$
R@N = \frac{1}{|\mathcal U|}\sum_{u\in\mathcal U} \frac{\left|Rel_u@N\right|}{\left| Rel_u\right|}.
$$

Fijamos un umbral de 3, y consideramos que un ítem es relevante cuando su rating *verdadero* sea mayor que el umbral. Por otra parte, para obtener las métricas "@N", ordenamos las predicciones de los elementos de test según la puntuación predicha, y nos quedamos con las $N$ más altas. Finalmente, notemos que en general $|Rec_u@N|=N$, salvo que no haya suficientes predicciones para ese usuario en concreto.

In [18]:
def compute_precision_recall(y, y_hat, N, threshold=3):
    """Each column in y_hat must have at least one prediction."""
    n_items, n_users = y_hat.shape
    P_users = np.zeros(n_users)
    R_users = np.zeros(n_users)

    for u in range(n_users):
        user_ratings = [(y_hat[x, u], y[x, u]) for x in range(n_items)]

        # Sort user ratings by predicted value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum([true_rating > threshold
                     for _, true_rating in user_ratings])

        # Number of recommended items in top N
        n_rec_N = sum([pred > 0 for pred, _ in user_ratings[:N]])

        # Number of relevant items in top N
        n_rel_N = sum([true_rating > threshold
                       for _, true_rating in user_ratings[:N]])

        # Precision@N: Proportion of recommended items that are relevant
        P_users[u] = n_rel_N/n_rec_N if n_rec_N != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        R_users[u] = n_rel_N/n_rel if n_rel != 0 else 0

    # Average accross all users
    P = np.mean(P_users)
    R = np.mean(R_users)

    return P, R

Elegimos por ejemplo $k=20$ en el modelo, y calculamos las métricas $P$, $R$ y $F$ (media armónica) a distintos valores de $N$.

In [19]:
model = ItemKNN(k=20)
model.fit(X_train)
y_hat = model.predict(X_test)

Ns = [1, 3, 5, 10, 25, 50, 100]
for N in Ns:
    P, R = compute_precision_recall(y_test, y_hat, N, threshold=3)
    F = 2*P*R/(P + R) if P + R > 0 else 0
    print(f"P@{N}: {P:.4f}, R@{N}: {R:.4f}, F@{N}: {F:.4f}")

P@1: 0.6897, R@1: 0.0739, F@1: 0.1335
P@3: 0.6897, R@3: 0.2174, F@3: 0.3306
P@5: 0.6621, R@5: 0.3288, F@5: 0.4394
P@10: 0.6363, R@10: 0.6193, F@10: 0.6277
P@25: 0.6198, R@25: 0.9918, F@25: 0.7629
P@50: 0.6179, R@50: 1.0000, F@50: 0.7638
P@100: 0.6179, R@100: 1.0000, F@100: 0.7638


Observamos que la precisión desciende conforme aumenta el valor de $N$, aunque no de forma demasiado brusca. Además, parece que se estabiliza a partir de $N=25$ en un valor en torno a $0.62$.

En cuanto al recall, tiene el comportamiento opuesto: aumenta conforme aumentamos $N$. En este caso a partir de $N=25$ también se estabiliza en un valor de $1$. Esto se puede explicar observando que de media hay menos de 25 ítems relevantes (con puntuación mayor que 3) por cada usuario.

Aunque los mejores resultados en la métrica $F$ se obtengan con valores grandes de $N$, el valor de $F@10$ no es del todo malo, por lo que podemos decir que tenemos un recomendador top-10 decente.

Podemos probar como hicimos anteriormente a elegir un par de valores de $k$ y calcular la precisión y el recall para las dos métricas de similaridad consideradas. Estudiamos por ejemplo P@10 y R@10.

In [20]:
similarity = ['cosine', 'pearson']
ks = [10, 20]
for k in ks:
    for sim in similarity:
        model = ItemKNN(k=k, similarity=sim)
        model.fit(X_train)
        y_hat = model.predict(X_test)
        P, R = compute_precision_recall(y_test, y_hat, 10, threshold=3)
        F = 2*P*R/(P + R) if P + R > 0 else 0
        print(f"{k}-NN + {sim} --> P@10: {P:.4f}, R@10: {R:.4f}, F@10: {F:.4f}")

10-NN + cosine --> P@10: 0.6156, R@10: 0.5961, F@10: 0.6057
10-NN + pearson --> P@10: 0.6329, R@10: 0.6140, F@10: 0.6233
20-NN + cosine --> P@10: 0.6363, R@10: 0.6193, F@10: 0.6277
20-NN + pearson --> P@10: 0.6398, R@10: 0.6212, F@10: 0.6303


En este caso sucede al revés que con el MAE y el RMSE: obtenemos unos mejores resultados de evaluación con la métrica de Pearson.

### Ejercicio opcional (1)

*Utilizar librerías (por ejemplo, Mahout, RankSys, LibRec o LensKit en Java, MyMediaLite en C#, Surprise, LightFM o Elliot en Python, recommenderlab en R, u otra) sobre los mismos datos para generar las recomendaciones y comparar con los resultados anteriores. Contrastar los valores de MAE y RMSE de las diferentes pruebas.*

Para hacer este ejercicio he elegido la librería [Surprise](https://surprise.readthedocs.io/en/latest/) de Python. 

En primer lugar, cargamos los datos en el formato de columnas que acepta Surprise, pasándolos por un dataframe de Pandas.

In [21]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader

# Creation of the dataframes. Column names are irrelevant.
df_train = pd.read_csv(
    "data/training-ratings.dat",
    sep='\t',
    names=['userID', 'itemID', 'rating']
)
df_test = pd.read_csv(
    "data/test-ratings.dat",
    sep='\t',
    names=['userID', 'itemID', 'rating']
)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
trainset = Dataset.load_from_df(df_train, reader).build_full_trainset()
testset = Dataset.load_from_df(
    df_test, reader).build_full_trainset().build_testset()

Utilizamos el conjunto de entrenamiento para entrenar y el de test para predecir, calculando las métricas de MAE y RMSE. Utilizamos tanto la similaridad coseno como la similaridad Pearson, y fijamos el valor de $k=20$.

In [22]:
from surprise import accuracy
from surprise.prediction_algorithms.knns import KNNBasic

cosine_sim = {
    'name': 'cosine',
    'user_based': False  # compute similarities between items
}
pearson_sim = {
    'name': 'pearson',
    'user_based': False  # compute similarities between items
}

algorithms = [
    ("Cosine", KNNBasic(k=20, sim_options=cosine_sim)),
    ("Pearson", KNNBasic(k=20, sim_options=pearson_sim))
]

for name, alg in algorithms:
    # Train the algorithm on the trainset, and predict ratings for the testset
    alg.fit(trainset)
    print(f"-- {name} --")
    predictions = alg.test(testset)

    # Then compute error and evaluation metrics
    mae = accuracy.mae(predictions)
    rmse = accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
-- Cosine --
MAE:  0.8686
RMSE: 1.0794
Computing the pearson similarity matrix...
Done computing similarity matrix.
-- Pearson --
MAE:  0.8970
RMSE: 1.1023


Si recordamos nuestras puntuaciones al programar nosotros el algoritmo, vemos que están relativamente cerca de estos valores (sobre todo en el caso del coseno). Por tanto, podemos decir que nuestro algoritmo funciona bastante bien.

### Ejercicio opcional (2)

*Repetir alguno de los ejercicios anteriores con alguna colección de prueba (p.e., MovieLens).*

Repetimos el ejercicio opcional (1) con la colección [MovieLens](https://grouplens.org/datasets/movielens/), que ya viene integrada en el paquete Surprise. Para ilustrar los resultados utilizamos la versión con 100k ratings. Además, en vez de separar manualmente los conjuntos de entrenamiento y test, hacemos 5-fold cross validation.

In [23]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed),
movielens = Dataset.load_builtin('ml-100k')

for name, alg in algorithms:
    print(f"--- {name} --- ")
    # Run 5-fold cross-validation and print results
    cross_validate(alg, movielens, n_jobs=-1, 
                   measures=['MAE', 'RMSE'], cv=5, 
                   verbose=True)

--- Cosine --- 
Evaluating MAE, RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.8334  0.8302  0.8327  0.8258  0.8257  0.8296  0.0033  
RMSE (testset)    1.0520  1.0498  1.0523  1.0475  1.0417  1.0487  0.0039  
Fit time          5.15    4.38    4.79    4.75    3.29    4.47    0.64    
Test time         7.69    8.98    8.46    8.60    4.74    7.69    1.53    
--- Pearson --- 
Evaluating MAE, RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.8556  0.8553  0.8549  0.8608  0.8564  0.8566  0.0021  
RMSE (testset)    1.0628  1.0664  1.0651  1.0755  1.0684  1.0676  0.0043  
Fit time          5.93    5.56    5.64    5.61    2.71    5.09    1.20    
Test time         6.34    6.41    6.56    6.37    2.42    5.62    1.60    


En este caso vemos que para ambas métricas obtenemos unos errores medios de CV más pequeños que los que teníamos con nuestro pequeño conjunto de prueba.